# Writing the bucket

This notebook contains code for writing out the zarrs to a temporary bucket, which we then sync over to the public one.

Right now I am only writing the smaller datasets. We start running out of memory with the larger ones, which I should put some effort into addressing.

In [1]:
import cellxgene_census
import anndata as ad
import zarr

from pathlib import Path

import pandas as pd, numpy as np

/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(


In [2]:
census = cellxgene_census.open_soma(census_version="2024-07-01")

In [3]:
obs = census["census_data"]["homo_sapiens"]["obs"].read().concat().to_pandas()

In [4]:
dset_counts = obs["dataset_id"].value_counts()
dset_counts

dataset_id
f7c1c579-2dc0-47e2-ba19-8165c5a0e353    4062980
8e10f1c4-8e98-41e5-b65f-8cd89a887122    2480956
9f222629-9e39-47d0-b83f-e08d610c7479    2282447
55003f67-c494-46f1-83fb-902745646379    2122065
d6505c89-c43d-4c28-8c4f-7351a5fd5528    1775529
                                         ...   
a65bcc2d-4243-44c1-a262-ab7dcddfcf86        810
703f00e6-b996-48e5-bc34-00c41b9876f4        649
de17ac25-550a-4018-be75-bbb485a0636e        395
00ff600e-6e2e-4d76-846f-0eec4f0ae417        363
0895c838-e550-48a3-a777-dbcd35d30272        146
Name: count, Length: 678, dtype: int64

In [5]:
census_hs = census["census_data"]["homo_sapiens"]

In [6]:
def write_anndata_from_census(dataset_id: str, destination_directory: str):
    from anndata.experimental import write_elem
    from tiledbsoma import AxisQuery

    census = cellxgene_census.open_soma(census_version="2024-07-01") 

    adata = (
        census["census_data"]["homo_sapiens"]
        .axis_query(
            measurement_name="RNA",
            obs_query=AxisQuery(value_filter=f"dataset_id == '{dataset_id}'")
        )
        .to_anndata(X_name="normalized")
    )
    adata.var = adata.var.set_index("feature_name", drop=True)

    print(f"retrieved {dataset_id}")

    z = zarr.open(f"{destination_directory}/{dataset_id}.zarr", "w")
    write_elem(z, "/", adata)
    write_elem(z, "layers/normalized_csc", adata.X.tocsc())
    del adata
    zarr.consolidate_metadata(z.store)

    print(f"wrote {dataset_id}")

In [7]:
import zarr
from functools import partial
from dask import delayed
from dask.distributed import Client, LocalCluster

In [8]:
cluster = LocalCluster(threads_per_worker=1, n_workers=4, memory_limit="50 GiB")
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 4,Total memory: 200.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:37729,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 4
Started: Just now,Total memory: 200.00 GiB
Comm: tcp://127.0.0.1:43705,Total threads: 1
Dashboard: http://127.0.0.1:46241/status,Memory: 50.00 GiB
Nanny: tcp://127.0.0.1:43205,


In [9]:
write_dataset = partial(write_anndata_from_census, destination_directory="s3://tmp-census-zarr/v2/homo_sapiens/")

In [10]:
smallish_dsets = dset_counts.index[dset_counts < 1_000_000]
smallish_dsets

CategoricalIndex(['fd072bc3-2dfb-46f8-b4e3-467cb3223182',
                  '232f6a5a-a04c-4758-a6e8-88ab2e3a6e69',
                  'b165f033-9dec-468a-9248-802fc6902a74',
                  '65badd7a-9262-4fd1-9ce2-eb5dc0ca8039',
                  'ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3',
                  '0ba636a1-4754-4786-a8be-7ab3cf760fd6',
                  '842c6f5d-4a94-4eef-8510-8c792d1124bc',
                  '1a38e762-2465-418f-b81c-6a4bce261c34',
                  '2c820d53-cbd7-4e0a-be7a-a0ad1989a98f',
                  'c7775e88-49bf-4ba2-a03b-93f00447c958',
                  ...
                  '81e91ff8-f619-4ad1-a0c3-b45e1dc63f68',
                  'df287f8d-f50d-4620-ab96-489d559e6adc',
                  '4993d61c-1d04-4630-9c61-8d9431f39adc',
                  'b25f3834-69b3-4d87-a272-3938432d1f30',
                  '0c9a8cfb-6649-4d52-b418-6d8e56bd7afe',
                  'a65bcc2d-4243-44c1-a262-ab7dcddfcf86',
                  '703f00e6-b996-48e5-bc34-00c41b9

In [11]:
futures = client.map(write_dataset, list(smallish_dsets)[::-1])

In [12]:
from time import sleep

In [13]:
while len(futures):
    next_futures = []

    for f in futures:
        if f.status == "finished":
            f.release()
        else:
            next_futures.append(f)
    
    futures = next_futures
    sleep(30)

/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/github/anndata/src/anndata/experimental/__init__.py:44: FutureWarning: Importing write_elem from `annda

retrieved 00ff600e-6e2e-4d76-846f-0eec4f0ae417
retrieved de17ac25-550a-4018-be75-bbb485a0636e


<frozen abc>:106: FutureWarning: xarray subclass Dataset2D should explicitly define __slots__
<frozen abc>:106: FutureWarning: xarray subclass Dataset2D should explicitly define __slots__


retrieved 0895c838-e550-48a3-a777-dbcd35d30272
retrieved 703f00e6-b996-48e5-bc34-00c41b9876f4


KeyboardInterrupt: 

In [33]:
# Alternative approach
# write_tasks = [delayed(write_anndata_from_census(x)) for x in smallish_dsets]
# client.compute(write_tasks)

[<Future: pending, key: write_anndata_from_census-0a6b4670-a139-44bd-84fe-5a0848efff5b>,
 <Future: pending, key: write_anndata_from_census-5337d834-e3c5-448f-8303-6f46a72caee8>,
 <Future: pending, key: write_anndata_from_census-11b924b0-aaba-425f-8cea-df6766837a35>,
 <Future: pending, key: write_anndata_from_census-bd9a105b-b138-45ff-bdb2-ad84d4ce0d79>,
 <Future: pending, key: write_anndata_from_census-ff999009-823c-4990-bfb9-9009d0afd57c>,
 <Future: pending, key: write_anndata_from_census-96ccb47c-20cf-449e-bd79-16c3c8c7c6bc>,
 <Future: pending, key: write_anndata_from_census-318dfbd3-87ba-4d95-b8fd-4d7acc010624>,
 <Future: pending, key: write_anndata_from_census-73e9e921-5d7d-4e61-bd15-cb3088a1d709>,
 <Future: pending, key: write_anndata_from_census-753de108-6f3c-4312-ad53-04b834c42f77>,
 <Future: pending, key: write_anndata_from_census-6ffab82a-8845-45b2-9b32-1ee0982ccad8>,
 <Future: pending, key: write_anndata_from_census-0cd5cf48-2666-4755-af40-539e053d6523>,
 <Future: pending, ke

/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(
/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(


retrieved 04c0bdd8-840f-488e-b72d-2a1229bc65db
retrieved 9f499d32-400d-4c42-ac9a-fb1481844fee
retrieved d4e69e01-3ba2-4d6b-a15d-e7048f78f22e


2024-09-17 01:21:36,004 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 21.68 GiB -- Worker memory limit: 30.96 GiB


retrieved ebc2e1ff-c8f9-466a-acf4-9d291afaf8b3


...

retrieved 574e9f9e-f8b4-41ef-bf19-89a9964fd9c7


/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/tiledb/cloud/config.py:96: UserWarning: You must first login before you can run commands. Please run tiledb.cloud.login.
  warnings.warn(


retrieved 04c0bdd8-840f-488e-b72d-2a1229bc65db
wrote 5bc23cb2-8cfc-4960-bab1-c3027fbeefc4
wrote 8f48c031-6692-4c82-b6c2-e4ad35cbc0aa
retrieved e6b2ce27-681b-4409-a053-2681875936e5
wrote 574e9f9e-f8b4-41ef-bf19-89a9964fd9c7
retrieved 389bfbb9-8ef1-4582-8c41-410131c3d0eb
wrote 04c0bdd8-840f-488e-b72d-2a1229bc65db
retrieved 105c7dad-0468-4628-a5be-2bb42c6a8ae4
retrieved 2a156c92-c62f-4a51-b340-9df09caa589c
wrote e6b2ce27-681b-4409-a053-2681875936e5
wrote 105c7dad-0468-4628-a5be-2bb42c6a8ae4
wrote 389bfbb9-8ef1-4582-8c41-410131c3d0eb
retrieved 1e5bd3b8-6a0e-4959-8d69-cafed30fe814
wrote 2a156c92-c62f-4a51-b340-9df09caa589c
retrieved bcdec5fa-a7fa-4806-92bc-0cd02f40242f
retrieved d0ea3ec4-0f3b-4649-9146-1c0b5f303a55
retrieved 6dafb698-7a53-4699-ad13-1b5e2c164be7
wrote bcdec5fa-a7fa-4806-92bc-0cd02f40242f
wrote 1e5bd3b8-6a0e-4959-8d69-cafed30fe814
wrote 6dafb698-7a53-4699-ad13-1b5e2c164be7
wrote d0ea3ec4-0f3b-4649-9146-1c0b5f303a55
retrieved 11a3244a-6b2f-43ca-95a5-d2cd95c482d2
retrieved 9ec3

2024-09-17 01:49:26,955 - distributed.worker.memory - WARNING - Unmanaged memory use is high. This may indicate a memory leak or the memory may not be released to the OS; see https://distributed.dask.org/en/latest/worker-memory.html#memory-not-released-back-to-the-os for more information. -- Unmanaged memory: 21.87 GiB -- Worker memory limit: 30.96 GiB
2024-09-17 01:49:37,339 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 24.92 GiB -- Worker memory limit: 30.96 GiB
2024-09-17 01:49:37,537 - distributed.worker.memory - WARNING - Worker is at 79% memory usage. Resuming worker. Process memory: 24.71 GiB -- Worker memory limit: 30.96 GiB
2024-09-17 01:49:38,537 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 24.99 GiB -- Worker memory limit: 30.96 GiB
2024-09-17 01:49:40,448 - distributed.worker.memory - WARNING - Worker is at 77% memory usage. Resuming worker. Process memory: 2

wrote 489318a0-24c3-4f5c-b105-f084ed0ea026
wrote 32b9bdce-2481-4c85-ba1b-6ad5fcea844c
retrieved 9c4c8515-8f82-4c72-b0c6-f87647b00bbe
retrieved 2872f4b0-b171-46e2-abc6-befcf6de6306
wrote 7c1c3d47-3166-43e5-9a95-65ceb2d45f78
retrieved d4e69e01-3ba2-4d6b-a15d-e7048f78f22e
retrieved d288b10a-643e-44bd-b451-96e9588b2ee5
wrote 9c4c8515-8f82-4c72-b0c6-f87647b00bbe
wrote 2872f4b0-b171-46e2-abc6-befcf6de6306
retrieved 1e5bd3b8-6a0e-4959-8d69-cafed30fe814
retrieved b3be4624-0723-4f1d-846c-d31025da3718
wrote d288b10a-643e-44bd-b451-96e9588b2ee5
retrieved 182f6a56-7360-4924-a74e-1772e07b3031
wrote d4e69e01-3ba2-4d6b-a15d-e7048f78f22e
retrieved eeacb0c1-2217-4cf6-b8ce-1f0fedf1b569
wrote 1e5bd3b8-6a0e-4959-8d69-cafed30fe814
wrote b3be4624-0723-4f1d-846c-d31025da3718
retrieved d3566d6a-a455-4a15-980f-45eb29114cab
retrieved d41f45c1-1b7b-4573-a998-ac5c5acb1647
wrote 182f6a56-7360-4924-a74e-1772e07b3031
wrote eeacb0c1-2217-4cf6-b8ce-1f0fedf1b569
retrieved 9b188f26-c8e1-4a78-af15-622a35a371fc
retrieved 

In [1]:
import s3fs
import zarr
import dask.distributed as dd
from dask import delayed

client = dd.Client()
client

s3 = s3fs.S3FileSystem(anon=True)
stores = s3.ls("s3://tmp-census-zarr/v1/homo_sapiens")

def consolidate_store(store_uri):
    zarr.consolidate_metadata(zarr.open(store_uri).store)

client.compute([delayed(consolidate_store)(f"s3://{uri}") for uri in stores])

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 8
Total threads: 32,Total memory: 123.85 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:35351,Workers: 8
Dashboard: http://127.0.0.1:8787/status,Total threads: 32
Started: Just now,Total memory: 123.85 GiB
Comm: tcp://127.0.0.1:38937,Total threads: 4
Dashboard: http://127.0.0.1:37615/status,Memory: 15.48 GiB
Nanny: tcp://127.0.0.1:42379,


In [2]:
s3 = s3fs.S3FileSystem(anon=True)
stores = s3.ls("s3://tmp-census-zarr/v1/homo_sapiens")

In [3]:
def consolidate_store(store_uri):
    zarr.consolidate_metadata(zarr.open(store_uri).store)

In [4]:
stores

['tmp-census-zarr/v1/homo_sapiens/0041b9c3-6a49-4bf7-8514-9bc7190067a7.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00476f9f-ebc1-4b72-b541-32f912ce36ea.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00e5dedd-b9b7-43be-8c28-b0e5c6414a62.zarr',
 'tmp-census-zarr/v1/homo_sapiens/00ff600e-6e2e-4d76-846f-0eec4f0ae417.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01209dce-3575-4bed-b1df-129f57fbc031.zarr',
 'tmp-census-zarr/v1/homo_sapiens/0129dbd9-a7d3-4f6b-96b9-1da155a93748.zarr',
 'tmp-census-zarr/v1/homo_sapiens/015c230d-650c-4527-870d-8a805849a382.zarr',
 'tmp-census-zarr/v1/homo_sapiens/019c7af2-c827-4454-9970-44d5e39ce068.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01ad3cd7-3929-4654-84c0-6db05bd5fd59.zarr',
 'tmp-census-zarr/v1/homo_sapiens/01c93cf6-b695-4e30-a26e-121ae8b16a9e.zarr',
 'tmp-census-zarr/v1/homo_sapiens/02792605-4760-4023-82ad-40fc4458a5db.zarr',
 'tmp-census-zarr/v1/homo_sapiens/03181d87-4769-41e7-8c39-d9a81835f0d2.zarr',
 'tmp-census-zarr/v1/homo_sapiens/0374f03c-62e2-4859-8a14-acb00b

In [6]:
client.compute([delayed(consolidate_store)(f"s3://{uri}") for uri in stores])

[<Future: pending, key: consolidate_store-8253dec2-3df9-49c5-be67-c9bc5278429e>,
 <Future: pending, key: consolidate_store-dd4352b1-2754-47c7-8f9a-29a67c2e3205>,
 <Future: pending, key: consolidate_store-794dd0ac-c23a-469e-87b8-d50a11e593d3>,
 <Future: pending, key: consolidate_store-ab18140e-4602-4d5b-9911-70eafa2a1c87>,
 <Future: pending, key: consolidate_store-dc309974-ac50-4006-98f1-7cd73896ef57>,
 <Future: pending, key: consolidate_store-07f827df-bac2-4faf-b446-e78c506e1e0a>,
 <Future: pending, key: consolidate_store-90439415-1991-4e3e-b307-93e1519174ae>,
 <Future: pending, key: consolidate_store-858b2585-d6f6-4b76-b19d-3bd1e248c17f>,
 <Future: pending, key: consolidate_store-7758dbda-ce93-482d-a32d-cca0641923da>,
 <Future: pending, key: consolidate_store-e12ed407-a89e-40fe-8993-f58f7870d1f7>,
 <Future: pending, key: consolidate_store-342440de-8e45-4d38-ab96-c8d599df6cc1>,
 <Future: pending, key: consolidate_store-251fb6fd-a1af-4bd8-84b5-a58f94e0e07d>,
 <Future: pending, key: cons

2024-09-18 17:42:49,514 - distributed.worker - ERROR - Compute Failed
Key:       consolidate_store-f685e54e-d070-46cd-951e-f7f41f904d80
State:     executing
Function:  consolidate_store
args:      ('s3://tmp-census-zarr/v1/homo_sapiens/e47c65a8-7d2f-48b8-908e-04ea6505fa26.zarr')
kwargs:    {}
Exception: "KeyError('.zmetadata')"
Traceback: '  File "/tmp/ipykernel_22598/4208769998.py", line 2, in consolidate_store\n  File "/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/zarr/convenience.py", line 1295, in consolidate_metadata\n    store[metadata_key] = json_dumps(out)\n    ~~~~~^^^^^^^^^^^^^^\n  File "/home/ubuntu/miniforge3/envs/cellxgene-census-dev-new/lib/python3.11/site-packages/zarr/storage.py", line 1473, in __setitem__\n    raise KeyError(key) from e\n'

2024-09-18 17:42:49,535 - distributed.worker - ERROR - Compute Failed
Key:       consolidate_store-f5118bf7-d314-49ee-b4b7-9a34f546fc68
State:     executing
Function:  consolidate_store
args:    

In [3]:
import boto3

In [9]:
import s3fs
import zarr